In [1331]:
import pandas as pd
import numpy as np
import scipy.optimize
from random import sample
import sys

## Benchmark (single $\nu$)

### Run Stata do file

In [67]:
cd MR16_data/replication\ files/

/Users/mizuhirosuzuki/Dropbox/EconPapers/MR16_data/replication files


In [87]:
!stata-se -b do wage_gap_one_nu.do

In [838]:
data_df = pd.read_fwf('data.raw', header = None)
data_df.columns =['Pop', 'Class', 'MA', 'VA', 'Migration', 'notsure1', 'notsure2', 'nu', 'num_nomig', 'income_weight']
data_df.head(30)

,Pop,Class,MA,VA,Migration,notsure1,notsure2,nu,num_nomig,income_weight
0,156,0,0.032125,0.022618,0.000000,1,1,23.6203,156,0.946624
1,156,1,0.036373,0.023784,0.000000,1,1,23.6203,156,0.963888
2,155,2,0.037962,0.021379,0.006452,1,1,23.6203,154,0.928850
3,156,3,0.042747,0.022370,0.006410,1,1,23.6203,155,0.960291
4,155,4,0.057714,0.042549,0.012903,1,1,23.6203,153,1.000000
5,161,0,0.039269,0.026786,0.000000,3,1,23.6203,161,0.796490
6,161,1,0.047074,0.031024,0.000000,3,1,23.6203,161,0.844159
7,161,2,0.051562,0.027803,0.000000,3,1,23.6203,161,0.812060
8,161,3,0.054943,0.030272,0.000000,3,1,23.6203,161,0.854544
9,161,4,0.069600,0.039060,0.000000,3,1,23.6203,161,1.000000


In [840]:
Pop_array = data_df.Pop.values.reshape((5, 100), order = 'F')
MA_array  = data_df.MA.values.reshape((5, 100), order = 'F')
VA_array  = data_df.VA.values.reshape((5, 100), order = 'F')
num_nomig_array = data_df.num_nomig.values.reshape((5, 100), order = 'F')
income_weight_array = data_df.income_weight.values.reshape((5, 100), order = 'F')


### Structural estimation

In [1110]:
np.random.seed(123)
P = sample(range(50, 501, 50), 1)[0]
N = 5
MA_list = []
for i in range(N):
    MA_list.append(np.random.uniform(i / 4 - 4, (i + 1) / 4 - 4))
MA = np.exp(MA_list)
VA_list = []
for i in range(N):
    VA_list.append(np.random.uniform(i / 6 - 5, (i + 1) / 6 - 5))
VA = np.exp(VA_list)
RA = VA / (MA ** 2)

beta = 0.5
nu = 23.6

In [788]:
P = 50
N = 5
MA = data_df.MA[:5].to_numpy()
VA = data_df.VA[:5].to_numpy()
RA = VA / (MA ** 2)

beta = 1.4
nu = 23.6

In [844]:
print(MA)
print(VA)
print(RA)

[0.02179918 0.02526172 0.03195946 0.04450422 0.05959803]
[0.00723025 0.00937346 0.01054052 0.0120361  0.01401005]
[15.2150208  14.6883946  10.31960925  6.07691966  3.94435365]


In [845]:
income_weight = MA[:(N - 1)] / MA[N - 1]

In [846]:
N_part = np.repeat(P / N - 1, N)
Pop_class = np.repeat(P/ N, N)

If $\beta < 1$, then all households would send migrants to the city without insurance network.
Hence, total surplus from insurance network is

\begin{equation*}
    W = \sum_k \left[ P_k \int_0^{\epsilon_{Ik}} \left\{ \left[ \log(M_{Ik}) - \frac{1}{2} R_I \right] - \left[ \log(M_{Ak}) - \frac{1}{2} \beta R_{Ak} + \epsilon \right] \right\} f(\epsilon) d \epsilon \right].
\end{equation*}

Since $N_k = P_k \int_0^{\epsilon_{Ik}} f(\epsilon) d \epsilon$,

\begin{equation*}
    W = \sum_k \left[ N_k \epsilon_{Ik} - P_k \int_0^{\epsilon_{Ik}} \epsilon f(\epsilon) d \epsilon \right].
\end{equation*}

When $F(\epsilon) = 1 - \exp(-\nu \epsilon)$, the above expression becomes

\begin{equation*}
    W = \sum_k \left[ N_k \epsilon_{Ik} - P_k \left(\frac{1}{\nu} - \left(\epsilon_{Ik} + \frac{1}{\nu} \right) \exp(- \nu \epsilon_{Ik}) \right) \right].
\end{equation*}


On the other hand, if $\beta > 1$, then there exists a threshold $\epsilon_{Ak}$ below which household do not send migrants to the city even when they do not benefit from insurance network, due to high risks of urban jobs (footnote 23).
In this case, total surplus from insurance network is

\begin{equation*}
    W = \sum_k \left[ P_k \left[ \int_0^{\epsilon_{Ak}} \left\{ \left[ \log(M_{Ik}) - \frac{1}{2} R_I \right] - \left[ \log(M_{Ak}) - \frac{1}{2} R_{Ak} \right] \right\} f(\epsilon) d \epsilon + \int_{\epsilon_{Ak}}^{\epsilon_{Ik}} \left\{ \left[ \log(M_{Ik}) - \frac{1}{2} R_I \right] - \left[ \log(M_{Ak}) - \frac{1}{2} \beta R_{Ak} + \epsilon \right] \right\} f(\epsilon) d \epsilon \right] \right] .
\end{equation*}

The first integral is the surplus of those who would not send migrants even without insurance: utility under insurance net of utility under autarky in a village.
The second integral is the surplus of those who would send migrants without insurance network: utility under insurance net of utility of sending migrants to the city.

Letting $N_{Ak} = P_k \int_0^{\epsilon_{Ak}} f(\epsilon) d \epsilon$, we can simplify the total surplus to

\begin{equation*}
    W = \sum_k \left[ N_{k} \epsilon_{Ik} - N_{Ak} \epsilon_{Ak} - P_k \int_{\epsilon_{Ak}}^{\epsilon_{Ik}} \epsilon f(\epsilon) d \epsilon \right]  .
\end{equation*}

When $F(\epsilon) = 1 - \exp(-\nu \epsilon)$, the above expression becomes

\begin{equation*}
    W = \sum_k \left[ N_{k} \epsilon_{Ik} - N_{Ak} \epsilon_{Ak} - P_k \left( \left(\epsilon_{Ak} + \frac{1}{\nu} \right) \exp(- \nu \epsilon_{Ak}) - \left(\epsilon_{Ik} + \frac{1}{\nu} \right) \exp(- \nu \epsilon_{Ik}) \right) \right]  .
\end{equation*}


The threshold $\epsilon_{Ak}$ is determined from the following equation equating the utility of staying in autarky and that of sending migrants:

\begin{equation*}
    \log(M_{Ak}) - \frac{1}{2} \beta R_{Ak} + \epsilon_{Ak} = \log(M_{Ak}) - \frac{1}{2} R_{Ak}.
\end{equation*}

Hence, $\epsilon_{Ak} = \frac{\beta - 1}{2} R_{Ak}$.

### ===================================

In [1850]:
def fp_N_fun(N_part_trans, income_weight, Pop_class, MA, VA, N):
    N_part = 1e-16 + np.exp(np.minimum(N_part_trans, 50)) / (1 + np.exp(np.minimum(N_part_trans, 50))) * ((Pop_class - 1e-16) - 1e-16)

    RA = VA / np.square(MA)
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = np.square(income_weight / np.dot(income_weight, N_part)) * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / np.square(np.dot(N_part, MA))
    return((N_part / Pop_class - (1 - np.exp(np.minimum(- nu * (np.log(MI) - np.log(MA) + 1 / 2 * beta * RA - 1 / 2 * RI), 0)))))

In [1851]:
def calculate_W(income_weight_trans, beta, MA, VA, Pop_class, N = N):
    income_weight = MA[:(N - 1)] / MA[N - 1] + np.exp(np.minimum(income_weight_trans, 50)) / (1 + np.exp(np.minimum(income_weight_trans, 50))) * (1 - MA[:(N - 1)] / MA[N - 1])
    income_weight = np.append(income_weight, 1)

    fp_N_res = scipy.optimize.root(fp_N_fun, np.repeat(0, N), 
                                   args = (income_weight, Pop_class, MA, VA, N), method = 'hybr',
#                                   options = {'xtol': 1e-16}
                                  )
    if np.abs(fp_N_res.fun).sum() > 1e-6:
        fp_N_res = scipy.optimize.root(fp_N_fun, np.repeat(0, N), 
                                       args = (income_weight, Pop_class, MA, VA, N), method = 'df-sane',
    #                                   options = {'xtol': 1e-16}
                                      )        
    N_part = 1e-16 + np.exp(np.minimum(fp_N_res.x, 50)) / (1 + np.exp(np.minimum(fp_N_res.x, 50))) * ((Pop_class - 1e-16) - 1e-16)
    
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = np.square(income_weight / np.dot(income_weight, N_part)) * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / np.square(np.dot(N_part, MA))
    
    # total surplus
    epsilon_I = np.log(MI) - np.log(MA) + 1 / 2 * beta * RA - 1 / 2 * RI

    if beta <= 1:
        W = np.dot(N_part, epsilon_I) - np.dot(Pop_class, 1 / nu - (epsilon_I + 1 / nu) * np.exp(- nu * epsilon_I))
    else:
        epsilon_A = (beta - 1) / 2 * VA / np.square(MA)
        N_A = Pop_class * (1 - np.exp(- nu * epsilon_A))
        W = np.dot(N_part, epsilon_I) - np.dot(N_A, epsilon_A) - np.dot(Pop_class, (epsilon_A + 1 / nu) * np.exp(- nu * epsilon_A) - (epsilon_I + 1 / nu) * np.exp(- nu * epsilon_I))
    return(-W)

In [1854]:
lambda_N_res_array = np.zeros([(N - 1) + N, MA_array.shape[1]])
optimization_res = np.repeat('', MA_array.shape[1])
fp_N_res_array = np.zeros([N, MA_array.shape[1]])
fp_N_message_array = np.repeat("", MA_array.shape[1])
for i in range(MA_array.shape[1]):
    MA = MA_array[:, i]
    VA = VA_array[:, i]
    Pop_class = Pop_array[:, i]
    lambda_N_init = np.repeat(-0, (N - 1))
    lambda_N_res = scipy.optimize.minimize(calculate_W, lambda_N_init, 
                        args = (1.41, MA, VA, Pop_class),
                        options = {'maxiter': 1000},
#                         method = 'Nelder-Mead',
#                         constraints = [{'fun': constraint_nonnegative_lambda, 'type': 'ineq'},
#                                        {'fun': constraint_less_than_one_lambda, 'type': 'ineq'}]
                                      )
    optimization_res[i] = lambda_N_res.message
    
    income_weight = MA[:(N - 1)] / MA[N - 1] + np.exp(np.minimum(lambda_N_res.x, 50)) / (1 + np.exp(np.minimum(lambda_N_res.x, 50))) * (1 - MA[:(N - 1)] / MA[N - 1])
    fp_N_res = scipy.optimize.root(fp_N_fun, np.repeat(0, N), 
                                   args = (np.append(income_weight, 1), Pop_class, MA, VA, N), method = 'hybr',
                                  options = {'xtol': 1e-16}
                                  )
    if np.abs(fp_N_res.fun).sum() > 1e-6:
        fp_N_res = scipy.optimize.root(fp_N_fun, np.repeat(0, N), 
                                       args = (np.append(income_weight, 1), Pop_class, MA, VA, N), method = 'df-sane',
    #                                   options = {'xtol': 1e-16}
                                      )        

    fp_N_res_array[:,i] = fp_N_res.fun
    fp_N_message_array[i] = fp_N_res.message
    N_part = 1e-16 + np.exp(np.minimum(fp_N_res.x, 50)) / (1 + np.exp(np.minimum(fp_N_res.x, 50))) * ((Pop_class - 1e-16) - 1e-16)

    lambda_N_res_array[:, i] = np.append(income_weight, N_part)

/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp
/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/scipy/optimize/optimize.py:697: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp
/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp
/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/scipy/optimize/optimize.py:697: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/scipy/optimize/_spectral.py:144: RuntimeWarning: divide by zero encountered in double_scalars
  sigma_k = np.vdot(s_k, s_k) / np.vdot(s_k, y_k)
/Users/miz

In [1838]:
fp_N_message_array

array(['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'x', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T'], dtype='<U1')

In [1855]:
pd.DataFrame(fp_N_res_array[:,(np.abs(fp_N_res_array).sum(axis=0) > 1e-6)])

,0
0,1.395361e-07
1,7.518116e-04
2,5.000000e-18
3,5.263158e-18
4,5.263158e-18


In [1709]:
i = 40
pd.DataFrame(fp_N_res_array[:,i:(i+10)])

,0,1,2,3,4,5,6,7,8,9
0,-1.110223e-16,0.000000e+00,-3.048537e-02,0.000000e+00,0.000000e+00,0.000000e+00,-1.110223e-16,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000000e+00,-1.110223e-16,1.253744e-03,1.250000e-17,0.000000e+00,0.000000e+00,-1.110223e-16,3.041594e-17,4.471318e-17,0.000000e+00
2,0.000000e+00,0.000000e+00,3.010162e-16,2.330800e-13,0.000000e+00,0.000000e+00,-2.220446e-16,5.000012e-18,5.874033e-16,0.000000e+00
3,1.110223e-16,0.000000e+00,4.204923e-02,2.437411e-13,0.000000e+00,1.110223e-16,-1.110223e-16,3.875125e-16,1.667765e-15,-1.110223e-16
4,0.000000e+00,0.000000e+00,4.273034e-02,2.108354e-13,7.142857e-18,0.000000e+00,0.000000e+00,3.404750e-16,1.953723e-15,3.044150e-17


In [1710]:
pd.DataFrame(lambda_N_res_array[:,i:(i+10)])

,0,1,2,3,4,5,6,7,8,9
0,0.786877,0.658108,4.319214e-01,5.765246e-01,6.944513e-01,0.999503,0.615719,5.887121e-01,5.862424e-01,6.712444e-01
1,0.852825,0.658252,6.943830e-01,6.484008e-01,7.540268e-01,0.999672,0.618151,7.064293e-01,6.804045e-01,6.712442e-01
2,0.895457,0.658949,6.334953e-01,7.505213e-01,8.062249e-01,0.999887,0.621613,7.644828e-01,7.315265e-01,7.364428e-01
3,0.854144,0.658865,8.009470e-01,8.423159e-01,9.032258e-01,0.999923,0.633483,8.528031e-01,8.853607e-01,8.808466e-01
4,45.000000,27.000000,8.628674e+01,7.953306e+00,1.499254e+01,21.000000,11.000000,1.873127e+01,1.289142e+01,1.100000e+01
5,44.000000,26.000000,2.364706e+01,1.000000e-16,1.398727e+01,20.000000,11.000000,6.083188e-16,6.259845e-16,1.000000e+01
6,45.000000,25.999999,2.679044e-14,1.631560e-12,1.397500e+01,21.000000,10.000000,1.000002e-16,8.223646e-15,9.994442e+00
7,43.999990,26.000000,3.700332e+00,1.949929e-12,1.393603e+01,19.999959,11.000000,7.750249e-15,2.334872e-14,9.970621e+00
8,44.000000,25.818357,3.760270e+00,1.475848e-12,1.000000e-16,20.000000,9.973396,6.469025e-15,2.735212e-14,3.044150e-16


In [1651]:
optimization_res

array(['O', 'O', 'O', 'O', 'O', 'O', 'D', 'O', 'D', 'D', 'D', 'D', 'D',
       'O', 'D', 'D', 'D', 'O', 'D', 'O', 'O', 'D', 'D', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'D', 'D', 'D', 'D', 'O', 'O', 'O', 'D', 'D',
       'D', 'D', 'D', 'D', 'D', 'O', 'O', 'D', 'D', 'D', 'O', 'O', 'D',
       'O', 'O', 'O', 'D', 'D', 'O', 'D', 'O', 'D', 'O', 'D', 'O', 'O',
       'O', 'O', 'O', 'D', 'D', 'D', 'D', 'O', 'O', 'O', 'D', 'D', 'O',
       'D', 'O', 'D', 'O', 'O', 'D', 'D', 'O', 'O', 'O', 'O', 'O', 'O',
       'D', 'O', 'D', 'D', 'O', 'D', 'D', 'O', 'D'], dtype='<U1')

In [1653]:
np.sum(optimization_res == 'D')

48

In [1708]:
i = 40
pd.DataFrame((Pop_array[:,i:(i+10)] - lambda_N_res_array[4:,i:(i+10)]) / Pop_array[:,i:(i+10)])

,0,1,2,3,4,5,6,7,8,9
0,1.578984e-16,0.000000e+00,0.030486,0.005837,0.000497,0.000000,1.614870e-16,0.063436,0.140572,0.000000
1,0.000000e+00,6.285570e-15,0.731283,1.000000,0.000909,0.000000,1.614870e-16,1.000000,1.000000,0.000000
2,1.578984e-16,5.367400e-08,1.000000,1.000000,0.001785,0.000000,1.776357e-16,1.000000,1.000000,0.000556
3,2.327516e-07,2.447820e-12,0.957951,1.000000,0.004570,0.000002,1.614870e-16,1.000000,1.000000,0.002938
4,2.863513e-10,6.986261e-03,0.957270,1.000000,1.000000,0.000000,2.660435e-03,1.000000,1.000000,1.000000


In [1707]:

pd.DataFrame(MA_array[:,i:(i+10)])

,0,1,2,3,4,5,6,7,8,9
0,0.471827,0.420979,0.225040,0.000071,0.016205,0.737295,0.767586,0.029814,0.035364,0.031213
1,1.122641,0.816386,0.247031,0.028664,0.019818,1.715672,1.605638,0.051972,0.036812,0.039377
2,1.833535,1.378173,0.259167,0.054657,0.024722,2.198166,2.060626,0.100055,0.039707,0.050530
3,2.682802,1.883155,0.290291,0.074271,0.030171,2.932251,3.429799,0.118145,0.042805,0.077307
4,4.082834,2.750610,0.415330,0.138898,0.126306,4.155626,4.450112,0.699454,0.057760,0.854526


In [1822]:
def fp_N_fun(N_part_trans, income_weight, Pop_class, MA, VA, N):
    N_part = 1e-16 + np.exp(np.minimum(N_part_trans, 50)) / (1 + np.exp(np.minimum(N_part_trans, 50))) * ((Pop_class - 1e-16) - 1e-16)

    RA = VA / np.square(MA)
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = np.square(income_weight / np.dot(income_weight, N_part)) * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / np.square(np.dot(N_part, MA))
    return((N_part / Pop_class - (1 - np.exp(np.minimum(- nu * (np.log(MI) - np.log(MA) + 1 / 2 * beta * RA - 1 / 2 * RI), 0)))))

In [1823]:
def calculate_W(income_weight_trans, beta, MA, VA, Pop_class, N = N):
    income_weight = MA[:(N - 1)] / MA[N - 1] + np.exp(np.minimum(income_weight_trans, 50)) / (1 + np.exp(np.minimum(income_weight_trans, 50))) * (1 - MA[:(N - 1)] / MA[N - 1])
    income_weight = np.append(income_weight, 1)

    fp_N_res = scipy.optimize.root(fp_N_fun, np.repeat(0, N), 
                                   args = (income_weight, Pop_class, MA, VA, N), method = 'hybr',
                                  options = {'xtol': 1e-16}
                                  )
    print(fp_N_res.fun)
    N_part = 1e-16 + np.exp(np.minimum(fp_N_res.x, 50)) / (1 + np.exp(np.minimum(fp_N_res.x, 50))) * ((Pop_class - 1e-16) - 1e-16)
    
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = np.square(income_weight / np.dot(income_weight, N_part)) * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / np.square(np.dot(N_part, MA))
    
    # total surplus
    epsilon_I = np.log(MI) - np.log(MA) + 1 / 2 * beta * RA - 1 / 2 * RI

    if beta <= 1:
        W = np.dot(N_part, epsilon_I) - np.dot(Pop_class, 1 / nu - (epsilon_I + 1 / nu) * np.exp(- nu * epsilon_I))
    else:
        epsilon_A = (beta - 1) / 2 * VA / np.square(MA)
        N_A = Pop_class * (1 - np.exp(- nu * epsilon_A))
        W = np.dot(N_part, epsilon_I) - np.dot(N_A, epsilon_A) - np.dot(Pop_class, (epsilon_A + 1 / nu) * np.exp(- nu * epsilon_A) - (epsilon_I + 1 / nu) * np.exp(- nu * epsilon_I))
    return(-W)

In [1808]:
np.where((np.abs(fp_N_res_array).sum(axis=0) > 1e-6))

(array([22, 33, 42, 51, 62, 78, 80, 91, 94]),)

In [1830]:
i = 51
MA = MA_array[:, i]
VA = VA_array[:, i]
Pop_class = Pop_array[:, i]
lambda_N_init = np.repeat(-0, (N - 1))
lambda_N_res = scipy.optimize.minimize(calculate_W, lambda_N_init, 
                    args = (1.41, MA, VA, Pop_class),
                    options = {'maxiter': 1000},
#                         constraints = [{'fun': constraint_nonnegative_lambda, 'type': 'ineq'},
#                                        {'fun': constraint_less_than_one_lambda, 'type': 'ineq'}]
                                  )
sss = MA[:(N - 1)] / MA[N - 1] + np.exp(np.minimum(lambda_N_res.x, 50)) / (1 + np.exp(np.minimum(lambda_N_res.x, 50))) * (1 - MA[:(N - 1)] / MA[N - 1])

[-0.01960362 -0.01932347 -0.01919997  0.00115793  0.03653149]
[-0.01960364 -0.01932349 -0.01919999  0.00115791  0.03653149]
[-0.0196036  -0.01932345 -0.01919995  0.00115786  0.03653149]
[-0.01960362 -0.01932347 -0.01919997  0.00115792  0.03653149]
[-0.01960362 -0.01932347 -0.01919997  0.00115788  0.03653149]
[-0.01960362 -0.01932347 -0.01919997  0.00115793  0.03653149]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.66666667e-18
 6.66666667e-18]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.66666667e-18
 6.66666667e-18]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.66666667e-18
 6.66666667e-18]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.66666667e-18
 6.66666667e-18]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.66666667e-18
 6.66666667e-18]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.66666667e-18
 6.66666667e-18]
[-0.00928922 -0.009133   -0.0089016   0.00735038  0.04766997]
[-0.00928922 -0.009133   -0.0089016   0.00735038  0.04766997]
[-0.00930111 -0.00913896 -0.00889916

[-0.01748557 -0.01731543 -0.01690617  0.00442293  0.05307213]
[-0.01748557 -0.01731543 -0.01690617  0.00442292  0.05307213]
[-0.01748556 -0.01731542 -0.01690616  0.00442294  0.05307213]
[-0.01708674 -0.01691309 -0.01605509  0.00088052  0.06250356]
[-0.01959941 -0.0194123  -0.01880425  0.00741193  0.05677716]
[-0.01525515 -0.01509325 -0.01464295  0.00185121  0.05499362]
[-0.01578519 -0.01562391 -0.01520846  0.00297233  0.05394967]
[-0.01865438 -0.01847476 -0.01802728 -0.00367979  0.05365847]
[-0.0180265  -0.01785378 -0.01743255  0.00311011  0.05320102]
[-0.01265359 -0.01248911 -0.01210012  0.0027786   0.05344624]
[-0.01265359 -0.01248911 -0.01210012  0.0027786   0.05344624]
[-0.01265384 -0.01248936 -0.01210036  0.00277858  0.05344622]
[-0.01265825 -0.01249372 -0.0121046   0.002778    0.05344595]
[-0.01265603 -0.01249153 -0.01210247  0.00277822  0.05344609]
[-0.01265696 -0.01249245 -0.01210337  0.00277811  0.05344603]
[-0.01910071 -0.01890504 -0.01843044  0.00479867  0.05306171]
[-0.0152

In [1831]:
sss

array([0.51826335, 0.5230405 , 0.52956588, 0.54523364])

In [1832]:
fp_N_res = scipy.optimize.root(fp_N_fun, np.repeat(0, N), 
                               args = (np.append(sss, 1), Pop_class, MA, VA, N), method = 'df-sane',
#                               options = {'xtol': 1e-16, 'maxiter': 1000, 'factor':1}
                              )
print(fp_N_res)
N_part = 1e-16 + np.exp(np.minimum(fp_N_res.x, 50)) / (1 + np.exp(np.minimum(fp_N_res.x, 50))) * ((Pop_class - 1e-16) - 1e-16)
N_part

     fun: array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.60671186e-09,
       6.66666667e-18])
 message: 'successful convergence'
    nfev: 353
     nit: 118
 success: True
       x: array([ 115.9803347 ,  115.9803347 ,  115.9803347 ,  -19.19575149,
       -115.9803347 ])


array([1.60000000e+01, 1.50000000e+01, 1.60000000e+01, 6.91006779e-08,
       1.00000000e-16])

In [1645]:
ttt = Pop_class - np.array([9.62121E-07, 0.000614913, 0.002452125, 0.009021544, 0.043538069]) * Pop_class
fp_N_fun(ttt, np.append(test, 1), Pop_class, MA, VA, N)

array([-9.11057325e-04, -9.14977571e-04, -2.50917922e-03, -9.70382682e-04,
        9.96791607e-01])

In [1519]:
fp_N_res

    fjac: array([[-5.88780760e-01, -2.20515705e-04, -3.01365369e-03,
        -3.18102624e-02, -8.07660940e-01],
       [-4.12700848e-03, -9.99986108e-01,  1.30873216e-05,
         1.35418721e-04,  3.27621228e-03],
       [-3.31593668e-01,  2.15975274e-03, -9.11068494e-01,
         9.64696655e-03,  2.44749075e-01],
       [-4.08872207e-01,  2.65894535e-03,  2.28656213e-01,
        -8.19734159e-01,  3.29497637e-01],
       [-6.13342760e-01,  4.00014073e-03,  3.43018153e-01,
         5.71778745e-01,  4.23322892e-01]])
     fun: array([-7.02336123e-05, -4.04064718e-04, -5.97695076e-03, -9.88120929e-03,
       -7.24461890e-03])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 60
     qtf: array([ 0.00636307,  0.00038385,  0.00340741,  0.00433516, -0.01072526])
       r: array([-1.28140526e-04,  4.82389110e-06,  5.92498004e-03,  4.67181855e-02,
       -1.14105460e-08, -6.88201639e-04, -2.20263573e-05, -1.7301283

In [1518]:
(Pop_class - N_part) / Pop_class

array([7.02439517e-05, 6.74211166e-04, 9.68391991e-03, 4.89919733e-02,
       9.99999831e-01])

In [1590]:
c = fp_N_fun(fp_N_res.x, np.append(test, 1), Pop_class, MA, VA, N)
c

array([-0.94366382, -0.94366382, -0.94366382, -0.94366382,  3.43352967])

In [1491]:
test = - np.log((1 - np.array([0.816417389, 0.815886261, 0.817099392, 0.828402354])) / (np.array([0.816417389, 0.815886261, 0.817099392, 0.828402354]) - MA[:(N-1)] / MA[N-1]))

array([1.17906803, 1.00710903, 0.93455465, 0.93380064])

In [1531]:
ttt = Pop_class - np.array([9.62121E-07, 0.000614913, 0.002452125, 0.009021544, 0.043538069]) * Pop_class
ttt

array([7.9999923 , 6.99569561, 7.980383  , 6.93684919, 6.69523352])

In [1524]:

sss = - np.log((Pop_class - 1e-16 - ttt) / (ttt - 1e-16))
fp_N_fun(sss, np.append(test, 1), Pop_class, MA, VA, N)

array([-9.60934490e-07, -5.83911956e-04, -2.02672643e-03, -4.53333208e-03,
        7.03869819e-02])

In [1416]:
optimization_res

array(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], dtype='<U1')

### ===================================

In [918]:
def calculate_W(lambda_N, beta, MA, VA, Pop_class, N = N):
    income_weight = lambda_N[:(N - 1)]
    N_part = lambda_N[(N - 1):]
    income_weight = np.append(income_weight, 1)
    
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = (income_weight / np.dot(income_weight, N_part)) ** 2 * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / (np.dot(N_part, MA) ** 2)
    
    # total surplus
    epsilon_I = - 1 / nu * np.log(1 - N_part / Pop_class + 1e-6)
    if beta <= 1:
        W = np.dot(N_part, epsilon_I) - np.dot(Pop_class, 1 / nu - (epsilon_I + 1 / nu) * np.exp(- nu * epsilon_I))
    else:
        epsilon_A = (beta - 1) / 2 * VA / (MA ** 2)
        N_A = Pop_class * (1 - np.exp(- nu * epsilon_A))
        W = np.dot(N_part, epsilon_I) - np.dot(N_A, epsilon_A) - np.dot(Pop_class, (epsilon_A + 1 / nu) * np.exp(- nu * epsilon_A) - (epsilon_I + 1 / nu) * np.exp(- nu * epsilon_I))
    return(-W)

In [919]:
def constraint_fixed_point(lambda_N):
    income_weight = lambda_N[:(N - 1)]
    N_part = lambda_N[(N - 1):]
    income_weight = np.append(income_weight, 1)
    
    RA = VA / (MA ** 2)
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = (income_weight / np.dot(income_weight, N_part)) ** 2 * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / (np.dot(N_part, MA) ** 2)
    
    return(N_part / Pop_class - (1 - np.exp(- nu * (np.log(MI) - np.log(MA) + 1 / 2 * beta * RA - 1 / 2 * RI))))

In [920]:
def constraint_less_than_one_lambda(lambda_N):
    income_weight = lambda_N[:(N - 1)]
    
    return(1 - income_weight)

In [921]:
def constraint_nonnegative_lambda(lambda_N):
    income_weight = lambda_N[:(N - 1)]
    
    return(income_weight - MA[:(N - 1)] / MA[N - 1])

In [922]:
def constraint_plausible_participant(lambda_N):
    N_part = lambda_N[(N - 1):]
    
    return(Pop_class - N_part)

In [923]:
def constraint_nonnegative_participant(lambda_N):
    N_part = lambda_N[(N - 1):]
    
    return(N_part)

In [892]:
lambda_N_init = np.append(income_weight * 1.1, Pop_class - 1)
lambda_N_res = scipy.optimize.minimize(calculate_W, lambda_N_init, 
                        args = (beta, MA, VA, Pop_class),
                        options = {'maxiter': 500, 'ftol': 1e-16},
                        constraints = [{'fun': constraint_fixed_point, 'type': 'eq'},
                                       {'fun': constraint_nonnegative_lambda, 'type': 'ineq'},
                                       {'fun': constraint_nonnegative_participant, 'type': 'ineq'},
                                       {'fun': constraint_plausible_participant, 'type': 'ineq'},
                                       {'fun': constraint_less_than_one_lambda, 'type': 'ineq'}
                                      ])

In [893]:
income_weight * 1.1

array([0.40234725, 0.46625517, 0.58987525, 0.82141372])

In [894]:
lambda_N_res

     fun: -30.952751961358704
     jac: array([     0.        ,      0.        ,      0.        ,      0.        ,
       -42399.20920014, -42401.60882783, -42399.20920014, -42401.60882783,
       -42401.60882783])
 message: 'Optimization terminated successfully.'
    nfev: 33
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([ 0.53908092,  0.65388631,  0.69524211,  0.82141372, 12.        ,
       11.        , 12.        , 11.        , 11.        ])

In [926]:
np.copy(MA_array)

array([[3.212530e-02, 3.926940e-02, 1.327450e-02, 2.781740e-02,
        3.118540e-02, 3.498200e-02, 3.451910e-02, 9.093640e-02,
        3.259670e-02, 3.931850e-02, 5.665900e-02, 3.451870e-02,
        2.862620e-02, 4.693960e-02, 2.823370e-02, 3.245490e-02,
        3.046140e-02, 3.279740e-02, 4.410000e-09, 3.327150e-02,
        1.047982e-01, 2.655700e-02, 2.657430e-02, 2.314320e-02,
        3.632090e-02, 4.624380e-02, 8.725430e-02, 8.405150e-02,
        1.709350e-02, 3.370380e-02, 5.143760e-02, 2.679420e-02,
        2.750280e-02, 4.139298e-01, 3.320560e-02, 5.296869e-01,
        5.500730e-02, 3.100000e-05, 7.821701e-01, 2.481443e-01,
        3.146700e-02, 6.512420e-02, 4.718272e-01, 4.209786e-01,
        2.250396e-01, 7.130000e-05, 1.620460e-02, 7.372955e-01,
        7.675860e-01, 2.981360e-02, 3.536370e-02, 3.121300e-02,
        2.526285e-01, 4.717548e-01, 4.603530e-02, 3.939350e-02,
        1.352993e-01, 6.010780e-02, 5.205050e-02, 6.651750e-02,
        8.457530e-02, 4.792490e-02, 5.13

In [929]:
lambda_N_res_array = np.zeros([(N - 1) + N, MA_array.shape[1]])

In [936]:
lambda_N_res_array = np.zeros([(N - 1) + N, MA_array.shape[1]])
optimization_res = np.zeros(MA_array.shape[1])
for i in range(MA_array.shape[1]):
    MA = MA_array[:, i]
    VA = VA_array[:, i]
    Pop_class = Pop_array[:, i]
    lambda_N_init = np.append(MA[:(N - 1)] / MA[N - 1], Pop_class - 1)
    lambda_N_res = scipy.optimize.minimize(calculate_W, lambda_N_init, 
                        args = (1.41, MA, VA, Pop_class),
                        options = {'maxiter': 500, 'ftol': 1e-16},
                        constraints = [{'fun': constraint_fixed_point, 'type': 'eq'},
                                       {'fun': constraint_nonnegative_lambda, 'type': 'ineq'},
                                       {'fun': constraint_nonnegative_participant, 'type': 'ineq'},
                                       {'fun': constraint_plausible_participant, 'type': 'ineq'},
                                       {'fun': constraint_less_than_one_lambda, 'type': 'ineq'}
                                      ])
    optimization_res[i] = lambda_N_res.success
    lambda_N_res_array[:, i] = lambda_N_res.x

/Users/mizuhirosuzuki/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log
  if sys.path[0] == '':


In [940]:
optimization_res

array([1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.])

In [367]:
income_weight_res = np.append(lambda_N_res.x[:(N - 1)], 1)
N_part_res = lambda_N_res.x[(N - 1):]

In [370]:
MI = income_weight_res / np.dot(income_weight_res, N_part_res) * np.dot(N_part_res, MA)
VI = (income_weight_res / np.dot(income_weight_res, N_part_res)) ** 2 * np.dot(N_part_res, VA)
RI = np.dot(N_part_res, VA) / (np.dot(N_part_res, MA) ** 2)

print(MI)
print(VI)
print(RI)

[0.02179919 0.02526172 0.03195946 0.04450422 0.05959803]
[8.30545690e-06 1.11534382e-05 1.78517910e-05 3.46166739e-05
 6.20793037e-05]
0.0174776488821353


In [277]:
N_part_res / Pop_class

array([0.99999996, 0.99999996, 0.99999996, 0.99999996, 0.99999996])

In [ ]:
sim_data = pd.DataFrame({'Pop': Pop_class,
                        'Class': range(N),
                        'MA': MA,
                        'VA': VA,
                        'nu': np.repeat(nu, N),
                        })

In [147]:
print(data_df[data_df.Class == 0].MA.mean(),
    data_df[data_df.Class == 1].MA.mean(),
    data_df[data_df.Class == 2].MA.mean(),
    data_df[data_df.Class == 3].MA.mean(),
    data_df[data_df.Class == 4].MA.mean())

print(data_df[data_df.Class == 0].MA.std(),
    data_df[data_df.Class == 1].MA.std(),
    data_df[data_df.Class == 2].MA.std(),
    data_df[data_df.Class == 3].MA.std(),
    data_df[data_df.Class == 4].MA.std())

print(data_df[data_df.Class == 0].VA.mean(),
    data_df[data_df.Class == 1].VA.mean(),
    data_df[data_df.Class == 2].VA.mean(),
    data_df[data_df.Class == 3].VA.mean(),
    data_df[data_df.Class == 4].VA.mean())

print(data_df[data_df.Class == 0].VA.std(),
    data_df[data_df.Class == 1].VA.std(),
    data_df[data_df.Class == 2].VA.std(),
    data_df[data_df.Class == 3].VA.std(),
    data_df[data_df.Class == 4].VA.std())


0.0950039413441 0.17183944596580006 0.2420777236928001 0.3296036684240001 0.5145493322089998
0.15747270902302024 0.35168540601355713 0.5152704429578014 0.7361840803337188 1.0640387980211676
0.04074292247037201 0.05010666158014501 0.050623472868269966 0.05709359035910002 0.115949197872
0.060199696417541464 0.08456809633669621 0.08111210573895539 0.09521975827285432 0.15524086889135344


In [125]:
np.exp(MA_list)

array([0.17070069, 0.20777762, 0.28430258, 0.44209592, 0.65256556])

In [126]:
np.exp(VA_list)

array([0.05534199, 0.0816912 , 0.0974133 , 0.11886498, 0.14927417])

In [149]:
MA_list = []
for i in range(N):
    MA_list.append(np.random.uniform(i / 4 - 2, (i + 1) / 4 - 2))
MA = np.exp(MA_list)
VA_list = []
for i in range(N):
    VA_list.append(np.random.uniform(i / 4 - 3, (i + 1) / 4 - 3))
VA = np.exp(VA_list)
RA = VA / (MA ** 2)

print(MA)
print(VA)
print(RA)

[0.15859528 0.2148872  0.26743301 0.3337897  0.44070077]
[0.0539736  0.06997951 0.08690548 0.11342976 0.15845954]
[2.14585752 1.51547855 1.2151143  1.01807822 0.81588843]


In [564]:
data_df.VA / (data_df.MA ** 2)

0      21.915635
1      17.978036
2      14.834996
3      12.241849
4      12.774074
         ...    
495    29.958330
496    10.912493
497     5.782403
498     5.300033
499     0.565970
Length: 500, dtype: float64

In [199]:
MA = np.array([data_df[data_df.Class == 0].MA.mean(),
            data_df[data_df.Class == 1].MA.mean(),
            data_df[data_df.Class == 2].MA.mean(),
            data_df[data_df.Class == 3].MA.mean(),
            data_df[data_df.Class == 4].MA.mean()])

VA = np.array([data_df[data_df.Class == 0].VA.mean(),
            data_df[data_df.Class == 1].VA.mean(),
            data_df[data_df.Class == 2].VA.mean(),
            data_df[data_df.Class == 3].VA.mean(),
            data_df[data_df.Class == 4].VA.mean()])
RA = VA / (MA ** 2)

print(MA)
print(VA)
print(RA)

[0.09500394 0.17183945 0.24207772 0.32960367 0.51454933]
[0.04074292 0.05010666 0.05062347 0.05709359 0.1159492 ]
[4.51407668 1.69687457 0.86385784 0.52553698 0.43793909]


##### ========================

In [941]:
def fp_N_fun(N_part, income_weight, Pop_class, MA, VA, N):
    
    RA = VA / (MA ** 2)
    # variables under insurance
    MI = income_weight / np.dot(income_weight, N_part) * np.dot(N_part, MA)
    VI = (income_weight / np.dot(income_weight, N_part)) ** 2 * np.dot(N_part, VA)
    RI = np.dot(N_part, VA) / (np.dot(N_part, MA) ** 2)
    print(MI)
    print(MA)
    print(RA)
    print(RI)
    return((N_part / Pop_class - (1 - np.exp(- nu * (np.log(MI) - np.log(MA) + 1 / 2 * beta * RA - 1 / 2 * RI)))))

In [942]:
fp_N_fun(aaa.x, np.append(income_weight,1), Pop_class, MA, VA, N)

[0.05149891 0.05967888 0.07550178 0.10513782 0.1407958 ]
[0.0176976 0.0274405 0.0430242 0.0905195 0.2539315]
[29.95832962 10.91249256  5.78240253  5.30003254  0.56596956]
0.00578268898496053


array([1.32857143e+01, 1.32857143e+01, 1.56666667e+01, 1.32857143e+01,
       4.20906471e+04])